### Using pyspark to create a logistic regression model to predict whether a customer will churn or not. 

By: Matt Purvis

In [0]:
from pyspark.sql import SparkSession

In [0]:
spark = SparkSession.builder.appName('Churn').getOrCreate()

In [0]:
# Load in the data using Spark SQL
df = spark.sql('select * from customer_churn_csv')

In [0]:
# Print the schema to get column names and datatypes
df.printSchema()

root
-- Names: string (nullable = true)
-- Age: double (nullable = true)
-- Total_Purchase: double (nullable = true)
-- Account_Manager: integer (nullable = true)
-- Years: double (nullable = true)
-- Num_Sites: double (nullable = true)
-- Onboard_date: string (nullable = true)
-- Location: string (nullable = true)
-- Company: string (nullable = true)
-- Churn: integer (nullable = true)

In [0]:
# Preview the dataset
df.show()

+-------------------+----+--------------+---------------+-----+---------+-------------------+--------------------+--------------------+-----+
 Names| Age|Total_Purchase|Account_Manager|Years|Num_Sites| Onboard_date| Location| Company|Churn|
+-------------------+----+--------------+---------------+-----+---------+-------------------+--------------------+--------------------+-----+
 Cameron Williams|42.0| 11066.8| 0| 7.22| 8.0|2013-08-30 07:00:40|10265 Elizabeth M...| Harvey LLC| 1|
 Kevin Mueller|41.0| 11916.22| 0| 6.5| 11.0|2013-08-13 00:38:46|6157 Frank Garden...| Wilson PLC| 1|
 Eric Lozano|38.0| 12884.75| 0| 6.67| 12.0|2016-06-29 06:20:07|1331 Keith Court ...|Miller, Johnson a...| 1|
 Phillip White|42.0| 8010.76| 0| 6.71| 10.0|2014-04-22 12:43:12|13120 Daniel Moun...| Smith Inc| 1|
 Cynthia Norton|37.0| 9191.58| 0| 5.56| 9.0|2016-01-19 15:31:15|765 Tricia Row Ka...| Love-Jones| 1|
 Jessica Williams|48.0| 10356.02| 0| 5.12| 8.0|2009-03-03 23:13:37|6187 Olson Mounta...| Kelly-Warren| 1|
 Eric Butler|44.0| 11331.58| 1| 5.23| 11.0|2016-12-05 03:35:43|4846 Savannah Roa...| Reynolds-Sheppard| 1|
 Zachary Walsh|32.0| 9885.12| 1| 6.92| 9.0|2006-03-09 14:50:20|25271 Roy Express...| Singh-Cole| 1|
 Ashlee Carr|43.0| 14062.6| 1| 5.46| 11.0|2011-09-29 05:47:23|3725 Caroline Str...| Lopez PLC| 1|
 Jennifer Lynch|40.0| 8066.94| 1| 7.11| 11.0|2006-03-28 15:42:45|363 Sandra Lodge ...| Reed-Martinez| 1|
 Paula Harris|30.0| 11575.37| 1| 5.22| 8.0|2016-11-13 13:13:01|Unit 8120 Box 916...|Briggs, Lamb and ...| 1|
 Bruce Phillips|45.0| 8771.02| 1| 6.64| 11.0|2015-05-28 12:14:03|Unit 1895 Box 094...| Figueroa-Maynard| 1|
 Craig Garner|45.0| 8988.67| 1| 4.84| 11.0|2011-02-16 08:10:47|897 Kelley Overpa...| Abbott-Thompson| 1|
 Nicole Olson|40.0| 8283.32| 1| 5.1| 13.0|2012-11-22 05:35:03|11488 Weaver Cape...|Smith, Kim and Ma...| 1|
 Harold Griffin|41.0| 6569.87| 1| 4.3| 11.0|2015-03-28 02:13:44|1774 Peter Row Ap...|Snyder, Lee and M...| 1|
 James Wright|38.0| 10494.82| 1| 6.81| 12.0|2015-07-22 08:38:40|45408 David Path ...| Sanders-Pierce| 1|
 Doris Wilkins|45.0| 8213.41| 1| 7.35| 11.0|2006-09-03 06:13:55|28216 Wright Moun...|Andrews, Adams an...| 1|
Katherine Carpenter|43.0| 11226.88| 0| 8.08| 12.0|2006-10-22 04:42:38|Unit 4948 Box 481...|Morgan, Phillips ...| 1|
 Lindsay Martin|53.0| 5515.09| 0| 6.85| 8.0|2015-10-07 00:27:10|69203 Crosby Divi...| Villanueva LLC| 1|
 Kathy Curry|46.0| 8046.4| 1| 5.69| 8.0|2014-11-06 23:47:14|9569 Caldwell Cre...|Berry, Orr and Ca...| 1|
+-------------------+----+--------------+---------------+-----+---------+-------------------+--------------------+--------------------+-----+
only showing top 20 rows

In [0]:
# Get list of column names
df.columns

Out[37]: ['Names',
 'Age',
 'Total_Purchase',
 'Account_Manager',
 'Years',
 'Num_Sites',
 'Onboard_date',
 'Location',
 'Company',
 'Churn']

In [0]:
# Filter the df down to columns we are interested in
my_cols = df.select([
 'Age',
 'Total_Purchase',
 'Years',
 'Num_Sites',
 'Churn'])

In [0]:
# Drop missing values - different strategies for handling missing data. We will drop for now. 
my_final_data = my_cols.na.drop()

In [0]:
# Import VectorAssembler
from pyspark.ml.feature import VectorAssembler

In [0]:
# Create assembler object that will transform the features into a vector - to be used in pipeline later
assembler = VectorAssembler(inputCols = ['Age','Total_Purchase','Years','Num_Sites'],
                           outputCol='features')

In [0]:
# import LogisticRegression class
from pyspark.ml.classification import LogisticRegression

In [0]:
# Import pipeline class
from pyspark.ml import Pipeline

In [0]:
# Create logistic regression model
log_reg_churn = LogisticRegression(featuresCol = 'features', labelCol = 'Churn')

In [0]:
# Create pipeline that will create the feature vector and the logistic regression model
pipeline = Pipeline(stages =  [assembler, log_reg_titanic])

In [0]:
# Split data into train and test - could take this further and create a validation set
train_data, test_data = my_final_data.randomSplit([.7,.3])

In [0]:
# Fit the pipeline to the training data
fit_model = pipeline.fit(train_data)

In [0]:
# Use pipeline to make same transformations on the test data
results = fit_model.transform(test_data)

In [0]:
# Import BinaryClassificationEvaluator to get the ROC
from pyspark.ml.evaluation import BinaryClassificationEvaluator

In [0]:
# Create evaluator object
my_eval = BinaryClassificationEvaluator(rawPredictionCol = 'prediction', labelCol = 'Churn')

In [0]:
# Preview the transformed test data
results.select('Churn', 'prediction').show()

+-----+----------+
Churn|prediction|
+-----+----------+
 0| 0.0|
 1| 0.0|
 0| 0.0|
 0| 0.0|
 1| 0.0|
 0| 0.0|
 0| 0.0|
 1| 0.0|
 0| 0.0|
 1| 0.0|
 0| 0.0|
 0| 0.0|
 0| 0.0|
 0| 0.0|
 0| 0.0|
 1| 0.0|
 0| 0.0|
 0| 0.0|
 0| 0.0|
 0| 1.0|
+-----+----------+
only showing top 20 rows

In [0]:
# Compare means and std devs of Churn and Predictions
results.select('Churn', 'prediction').describe().show()

+-------+-------------------+-------------------+
summary| Churn| prediction|
+-------+-------------------+-------------------+
 count| 269| 269|
 mean|0.14869888475836432|0.14869888475836432|
 stddev| 0.3564545804818795|0.35645458048187967|
 min| 0| 0.0|
 max| 1| 1.0|
+-------+-------------------+-------------------+

In [0]:
# Get the ROC/AUC to evaluate the model
AUC = my_eval.evaluate(results)

In [0]:
# Preview the ROC/AUC
AUC

Out[54]: 0.7797489082969432

A perfect model would have a 1.0 ROC/AUC. This model is not terrible but could be better!

### Predict on new data

In [0]:
# Read in unseen data
new_customers = spark.sql('select * from new_customers_csv')

In [0]:
# Preview the data
new_customers.show()

+--------------+----+--------------+---------------+-----+---------+-------------------+--------------------+----------------+
 Names| Age|Total_Purchase|Account_Manager|Years|Num_Sites| Onboard_date| Location| Company|
+--------------+----+--------------+---------------+-----+---------+-------------------+--------------------+----------------+
 Andrew Mccall|37.0| 9935.53| 1| 7.71| 8.0|2011-08-29 18:37:54|38612 Johnny Stra...| King Ltd|
Michele Wright|23.0| 7526.94| 1| 9.28| 15.0|2013-07-22 18:19:54|21083 Nicole Junc...| Cannon-Benson|
 Jeremy Chang|65.0| 100.0| 1| 1.0| 15.0|2006-12-11 07:48:13|085 Austin Views ...|Barron-Robertson|
Megan Ferguson|32.0| 6487.5| 0| 9.4| 14.0|2016-10-28 05:32:13|922 Wright Branch...| Sexton-Golden|
 Taylor Young|32.0| 13147.71| 1| 10.0| 8.0|2012-03-20 00:36:46|Unit 0789 Box 073...| Wood LLC|
 Jessica Drake|22.0| 8445.26| 1| 3.46| 14.0|2011-02-04 19:29:27|1148 Tina Straven...| Parks-Robbins|
+--------------+----+--------------+---------------+-----+---------+-------------------+--------------------+----------------+

In [0]:
# Make the same transformations using the pipeline on the new data
test_new_customers = fit_model.transform(new_customers)

In [0]:
# Look at predictions
test_new_customers.select(['Names','Age','Total_Purchase','Years','Num_Sites','Company','prediction']).show()

+--------------+----+--------------+-----+---------+----------------+----------+
 Names| Age|Total_Purchase|Years|Num_Sites| Company|prediction|
+--------------+----+--------------+-----+---------+----------------+----------+
 Andrew Mccall|37.0| 9935.53| 7.71| 8.0| King Ltd| 0.0|
Michele Wright|23.0| 7526.94| 9.28| 15.0| Cannon-Benson| 1.0|
 Jeremy Chang|65.0| 100.0| 1.0| 15.0|Barron-Robertson| 1.0|
Megan Ferguson|32.0| 6487.5| 9.4| 14.0| Sexton-Golden| 1.0|
 Taylor Young|32.0| 13147.71| 10.0| 8.0| Wood LLC| 0.0|
 Jessica Drake|22.0| 8445.26| 3.46| 14.0| Parks-Robbins| 1.0|
+--------------+----+--------------+-----+---------+----------------+----------+

Four of these customers are predicted to churn. We would want to assign account managers to these customers and give them extra care and attention in order to prevent them from churning.